In [2]:
import re
import gensim
import logging
import pandas as pd
import random
import os
import nltk
import pymorphy2
from gensim.models import word2vec
from tqdm import tqdm
# from pymystem3 import Mystem
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
# nltk.download('punkt')
from string import punctuation
from random import choice
morph = pymorphy2.MorphAnalyzer()
# model = w2v()


In [ ]:
Веб-сервис/телеграм-бот, который играет с пользователем в игру: 
предлагает угадать, оригинальное ли перед ним предложение какого-либо автора
или сгенерированное компьютером с помощью word2vec. 
# В конце показывает результат пользователя и статистику правильных ответов.

In [3]:
# работа с моделью (НКРЯ, 2015)


def w2v():
    # скачиваем модель
    # urllib.request.urlretrieve(
    #    "http://rusvectores.org/static/models/rusvectores2/\
    #    ruscorpora_mystem_cbow_300_2_2015.bin.gz",
    #    "ruscorpora_mystem_cbow_300_2_2015.bin.gz")
    m = r'C:\Users\annap\prog18\homeworks\hw7\ruscorpora_mystem_cbow_300_2_2015.bin.gz'
    if m.endswith('.vec.gz'):
        model = \
            gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
    elif m.endswith('.bin.gz'):
        model = \
            gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
    else:
        model = \
            gensim.models.KeyedVectors.load(m)
    return model

In [4]:
# замена тэгов pymorphy2 на mystem
pymorphyVSmystem = {
    'NOUN': 'S',
    'ADJF': 'A',
    'ADJS': 'A',
    'COMP': 'A',
    'VERB': 'V',
    'INFN': 'V',
    'PRTF': 'V',
    'PRTS': 'V',
    'GRND': 'V',
    'PRED': 'V',
    'ADVB': 'ADV',
    'NUMR': 'NUM',
    'NPRO': 'SPRO',
    'CONJ': 'CONJ',
    'PRCL': 'PART',
    'PREP': 'P',
    'INTJ': 'INTJ'
    }


In [5]:
# inflections = ['case', 'number', 'person', 'tense', 'mood', 'voice']
# словарь категорий слова, по которым будет изменяться похожее слово


def inflections(word, pos):
    inflection = {}
#     существительное не изменяется по роду
    if pos != 'NOUN':
        inflection['gender'] = word.tag.gender
    else:
        pass
    if pos == 'VERB':
        # у глагола с тэгом 'perf' отсутствует форма настоящего времени
        if word.tag.aspect != 'perf':
            inflection['tense'] = word.tag.tense
        else:
            pass
    inflection['case'] = word.tag.case
    inflection['number'] = word.tag.number
    inflection['person'] = word.tag.person
    inflection['mood'] = word.tag.mood
    inflection['voice'] = word.tag.voice
    return inflection


In [6]:
# список текстов из папки с текстами


def files():
    filelist = []
    start_path = r'.\texts'
    for filename in os.listdir(start_path):
        filepath = os.path.join(start_path, filename)
        filelist.append(filepath)
    return filelist


In [7]:
# случайно выбираем текст, из которого берем фразу


def name():
    name = random.choice(files())
    return name


In [8]:
# эта функция создает словарь (d)
# автор (Мураками или Компьютер): [предложение, название произведения],
# предварительно выбрав случайное предложение из текста
# (для интереса длиной > 50 знаков)


def sentSplit():
    d = {}
    all_sent_list = []
    sent_book = []
    filename = name()
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.readlines()
    for line in text:
        title = re.search(r'([а-яА-Я]|\s)+?\n', line)
        author = re.search('Харуки Мураками', line)
        date = re.search('\d\n', line)
        if not author and not date and not title:
            all_sent_list.extend(sent_tokenize(line))
    sentence = random.choice(all_sent_list)
    while len(sentence) < 50:
        sentence = random.choice(all_sent_list)
    sent_book.append(sentence)
    r1 = re.search(r'texts\\(.+?)\.txt', filename)
    if r1:
        sent_book.append(r1.group(1))
    d['Мураками'] = sent_book
    return d


In [9]:
# эта функция разбивает исходную строку на символы, чтобы потом можно было
# вернуть исходную пунктуацию


def rawString(phrase):
    symbols = []
    wordlist = []
    r2 = re.findall('(\w+)(\s)?([^\w\s])?(\s)?', phrase)
    for item in r2:
        symbols.append(item)
    for w in symbols:
        for i in w:
            if i != '':
                wordlist.append(i)
    return(wordlist)


In [10]:
# эта функция оставляет из предыдущего списка символов только слова


def onlyWords(wordlist):
    words = []
    for w in wordlist:
        r3 = re.search(r'[А-Яа-я]+', w)
        if r3:
            if w not in punctuation:
                words.append(w)
    return words


In [11]:
model = w2v()

c:\users\annap\miniconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [33]:
# эта функция создает словарь (origin_model)
# оригинальное слово: слово из модели


def wMDict(words, model):
    origin_model = {}
    word_info = {}
    normal_pos_infl = {}
    sw = stopwords.words('russian')
    for item in words:
        if item.lower() not in sw:
            pos_infl = {}
            word = morph.parse(item)[0]
            normal = word.normal_form
            pos = word.tag.POS
            pos_infl['pos'] = pos
            pos_infl['infl'] = inflections(word, pos)
            normal_pos_infl[normal] = pos_infl
            word_info.update(normal_pos_infl)
            if pymorphyVSmystem[normal_pos_infl[normal]['pos']]:
                query = normal + '_' +\
                pymorphyVSmystem[normal_pos_infl[normal]['pos']]
            else:
                pass
            if query in model:
                top5_similar = model.most_similar(positive=[query], topn=5)
                for word in top5_similar:
                    r4 = re.search('([а-я].+?)_(.+)', word[0])
                    if r4:
                        if r4.group(2) ==\
                              pymorphyVSmystem[normal_pos_infl[normal]['pos']]:
                            parsed = morph.parse(r4.group(1))[0]
                            inflection = word_info[normal]['infl'].values()
                            inflected = parsed.inflect({item for
                                                        item in inflection
                                                        if item is not None})
                            if inflected:
                                origin_model[item] = inflected.word
                                break
                            else:
                                origin_model[item] = item
                    else:
                        origin_model[item] = item
            else:
                origin_model[item] = item
    return origin_model


In [41]:
# меняем оригинальные слова на слова из модели, получаем строку,
# идентичную оригинальной, но со словами из модели


def wordChange(raw, origin_model):
    final = []
    for i in raw:
        try:
            final.append(origin_model[i])
        except:
            final.append(i)
    final[0] = final[0].capitalize()
    s = ''.join(final)
    return s


In [42]:
# эта функция создает словарь автор (компьютер): фраза, произведение
def comp():
    y = []
    computer = sentSplit()
    phrase = computer['Мураками'][0]
#     print(phrase)
    raw = rawString(phrase)
    ow = onlyWords(raw)
    origin_model = wMDict(ow, model)
    changed = wordChange(raw, origin_model)
#     print(changed)
    y.append(changed)
    y.append(computer['Мураками'][1])
    computer['Компьютер'] = y
    del computer['Мураками']
    return computer
# comp()

Порядок… Метки, показывавшие дорогу домой, тянулись за мной неровной чередой, как буйки на поверхности моря.
Распорядок… отметины, продемонстрировавшие шоссе обратно, протягивались за мной ровной вереницей, как буйки на коре океана.


{'Компьютер': ['Распорядок… отметины, продемонстрировавшие шоссе обратно, протягивались за мной ровной вереницей, как буйки на коре океана.',
  'Кафка на пляже']}

In [37]:
# список словарей автор: фраза, произведение,
# который будет использоваться в игре


def sentences():
    sent_list = []
    for i in range(0, 10):
        z = choice([sentSplit, comp])()
        sent_list.append(z)
    return sent_list


In [ ]:
# статистика!
# ваш ответ:

# правильный ответ:

In [38]:
sent_list = sentences()

print(sent_list)

[{'Мураками': ['В небе плавали белые облака, носились птицы и стрекозы.', 'Таиланд']}, {'Компьютер': ['На парковке стоял лишь блестящий синий «ниссан-блюберд»', 'Повторное нападение на булочную']}, {'Компьютер': ['Умная, сообразительная, могла бы отлично учиться\xa0— но принципиально держалась так, чтоб ее способностей никто не замечал.', '1Q84. Книга 3']}, {'Мураками': ['Ее шея пахла теперь просто как шея семнадцатилетней девушки.', 'Страна Чудес без тормозов и Конец Света']}, {'Мураками': ['—\xa0В машине вы обмолвились о медведе,\xa0— сказал Комура.', 'В Кусиро поселился НЛО']}, {'Компьютер': ['Нехорошо — чуть что обижать простодушных пиплов Северного Канто[4]сказал Кэйсукэ.', 'Пейзаж с утюгом']}, {'Компьютер': ['Белая береза”1910-1923) литературная группа, с деятельностью которой связаны имена писателей-неогуманистов Такэо Арисима, Наоя Сига, Санэацу Мусякодзи >', 'Мой любимый sputnik']}, {'Мураками': ['—\xa0Время и опыт,\xa0— повторила Сумирэ и посмотрела в небо.', 'Мой любимый spu

In [27]:
import flask
from flask import Flask
from flask import render_template, request, url_for, redirect

app = Flask(__name__)

@app.route('/')
def index():
    sent_list = sentences()
#     setattr(g, 'sent_list', None)
#     sent = getattr(flask.g, 'sent_list', None)
    flask.g = sent_list
    return render_template('index.html', sent_list=sent_list)

@app.route('/results')
def results():
    answrs = []
    correct = 0
#     flask.g.sent = 
#     sent_list = getattr(g, 'sent_list', None)
    sent_list = flask.g
    if request.args:
        for item in sent_list:
            for key, value in item.items():
                a = {}
                answer = request.args[value[0]]
                if answer == key:
                    correct += 1
                a[answer] = value[0]
                answrs.append(a)
        return render_template('results.html', correct=correct, rights=sent_list, users=answrs)
# + пройти заново
#     else:
#         return render_template('onemoretime.html')
#     return render_template('index.html', sent_list=sent_list)
    return redirect(url_for('index'))


if __name__ == '__main__':
    app.run(debug=False)

#     add numbered list
# подумать, как оставить один список в 2 функциях и при этом делать новые
# + дизайн

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jun/2019 06:31:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2019 06:32:07] "GET /results?В+палате+витал+сладковатый+запах+тушеной+рыбы%2C+оставшейся+нетронутой+после+обеда.=Компьютер&Сначала+поворачиваешь+его+так%2C+чтобы+он+глядел+на+тебя.=Мураками&+++Вернувшись+в+контору%2C+я+увидел%2C+что+дверь+открыта+настежь%2C+а+внутри+никого+нет.=Мураками&Проникавшая+в+окно+морось+вымочила+все+в+машине+—+сиденья%2C+лобовое+стекло%2C+ветровку%2C+сигареты+в+кармане...+Резко+взвыли+сирены+сухогрузов+на+рейде+—+так+голосят+отбившиеся+от+стада+телята.=Мураками&Аккуратно+расставлял+все+на+столе+и+неспешно+ел%2C+заодно+просматривая+газету.=Мураками&Давно+мне+не+приходилось+видеть+такого+крепкого+сна.=Мураками&Или+не+можете+успокоиться%2C+пока+кто-то+есть+рядом%3F=Мураками&Когда+я+вру%2C+с+моим+голосом+всегда+происходит+что-то+странное.=Мураками&В+зеркале+он+видит+лишь+себя+самого+–+того%2C+кто+он+в+реальности.=Мураками&Бо

In [150]:
def results():
    correct = 0
    if request.args:
        for item in listy:
#             answer = request.form[]
            for key, value in item.items():
                answer = request.args[value[0]]
#                 print(answer)
                if answer == key:
                    correct += 1
        return render_template('results.html', correct=correct)
    return render_template('index.html', listy=listy)

In [ ]:
@app.route('/results')
def results():
    correct = 0
    if request.args:
        for item in listy:
#             answer = request.form[]
            for key in item.items():
                answer = request.form[key]
                if key == answer:
                    correct += 1
    return render_template('results.html', correct=correct)


if __name__ == '__main__':
    app.run(debug=True)        
#     if request.args['']
    
#     или можно на все плюнуть и просто давать правильные ответы

In [ ]:
В конце показывает результат пользователя и статистику правильных ответов.!!!!!!
+ можно добавить комментарий

In [88]:
d = morph.parse('написать')[0]
print(d)
e = [None, None, 'plur', '2per', 'pres', 'indc', None]
f = d.inflect({'plur', '2per', 'indc', 'pres'})
# f = d.inflect({item for item in e if item is not None})
print(f)
# if perf - perf, if impf - pres

Parse(word='написать', tag=OpencorporaTag('INFN,perf,tran'), normal_form='написать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'написать', 916, 0),))
None


In [ ]:
для фласка:
    читаем по одному 

In [ ]:
заметочник:
    служебные слова оставляем
    числительные можем не трогать
    скопировать вход на выход - знаки препинания, заглавные буквы и тд - майстем(?)
    лемматизируем и сохраняем тэги слова
    чистые слова в модель - ближайшее слово
    это слово склоняем по характеристикам
    всталяем в "скелет"

                                                                                 
    